In [1]:
#!/usr/bin/python

import numpy as np
import math
from astropy.io import fits as fits

from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

import pandas as pd
from pandas import DataFrame, Series

from astropy.wcs import WCS as WCS

import import_ipynb
import Pipe3D_Asier_Test_IA as Pipe3D

from mpdaf.obj import hms2deg
from mpdaf.obj import dms2deg

from astropy.convolution import Gaussian2DKernel
from astropy.convolution import convolve

from scipy.stats import ks_2samp
from scipy import stats

/home/asier/anaconda2/lib/python2.7/site-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/home/asier/anaconda2/lib/python2.7/site-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/home/asier/anaconda2/lib/python2.7/site-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/home/asier/anaconda2/lib/python2.7/site-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/home/asier/anaconda2/lib/python2.7/site-packages/pandas/core/indexes/base.py:7: Ru

importing Jupyter notebook from Pipe3D_Asier_Test_IA.ipynb


/home/asier/anaconda2/lib/python2.7/site-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._solve_toeplitz import levinson
/home/asier/anaconda2/lib/python2.7/site-packages/scipy/linalg/__init__.py:207: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._decomp_update import *
/home/asier/anaconda2/lib/python2.7/site-packages/scipy/special/__init__.py:640: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ufuncs import *
/home/asier/anaconda2/lib/python2.7/site-packages/scipy/special/_ellip_harm.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ellip_harm_2 import _ellipsoid, _ellipsoid_norm
/home/asier/anaconda2/lib/python2.7/site-packages/scipy/signal/_max_len_seq.py:8: RuntimeWarning: numpy.dtype size changed, may ind

In [2]:
#Crear arrays con los datos de todas las galaxias.
Numb_Lines = len(open('Names_Amusing_true.txt').readlines())

All_SN_type = np.zeros(Numb_Lines)
All_Prob_Ha = np.zeros(Numb_Lines)
All_Cumul_Ha = np.zeros(Numb_Lines)

names = open('Names_Amusing_true.txt')

Galaxy = -1
nIa = 0
nII = 0
for lines in names:
    line= lines[:-1]
    print(line)
    Galaxy = Galaxy+1

    df = pd.read_csv('Amusing_Table_true.csv')  # To find the Galaxy data
    Names_1 = df['SN_name']
    Names_2 = df['def_name']
    R = 0

    for names in Names_1:              # Finding the Galaxy in the DataFrame
        R=R+1
        if line == names:
            Row = R-1
    R = 0
    for names in Names_2:              # Some Galaxies have the name in other column
        R=R+1
        if line == names:
            Row = R-1

    # SN coordinates
    RA_SN = df.at[Row,'RA_SN']
    RA_SN = hms2deg(RA_SN)
    
    DEC_SN = df.at[Row,'DEC_SN']
    DEC_SN = dms2deg(DEC_SN)
    
    # Galaxy center coordinates
    RA_GAL = df.at[Row,'RA_GAL']
    RA_GAL = hms2deg(RA_GAL)
    
    DEC_GAL = df.at[Row,'DEC_GAL']
    DEC_GAL = dms2deg(DEC_GAL)
    
    # Pixel position
    w = WCS('../Data/Amusing/flux_elines.'+line+'.cube.fits')
    Pix_X, Pix_Y ,z = w.wcs_world2pix(RA_SN, DEC_SN, 0. , 0,ra_dec_order=True)
    Pix_X_GAL, Pix_Y_GAL ,z = w.wcs_world2pix(RA_GAL, DEC_GAL, 0. , 0,ra_dec_order=True)

    # Integer for measure
    Pix_X = int(Pix_X)
    Pix_Y = int(Pix_Y)

    # Supernova type
    if df.at[Row,'SN_type'] == 'Ia':
        SN_TYPE = 1
        nIa = nIa + 1
    elif df.at[Row,'SN_type'] == 'II':
        SN_TYPE = 2
        nII = nII + 1
    else:
        SN_TYPE = 0

    #~~~~~~~~~~~~~~~~~~ H Alpha ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    # Call to Pipe3D code
    line = Pipe3D.Pipe3D_galaxy(line)

    # Read the Ha map of the galaxy
    hdulist = fits.open('../Data/Amusing/flux_elines.'+line.name+'.cube.fits')
    line.Ha_Map = hdulist[0].data[20]

    # Gaussian function
    stddev = 7
    gauss = Gaussian2DKernel(stddev)

    # Gaussian convolve
    line.Ha_Map = convolve(line.Ha_Map, gauss)

    # An Array whit all the value of the Ha map
    line.Prob_List = np.ndarray.flatten(line.Ha_Map)
    # The array orther in value
    line.Order_Prob_List = np.sort(line.Prob_List)

    # Standard deviation of negative values (indicatve of noise level)
    Sigmas_of_noise = 6

    noise_level = Sigmas_of_noise * np.nanstd(line.Prob_List[line.Prob_List<0])
    line.Order_Prob_List[line.Order_Prob_List<noise_level] = 0.
    line.Ha_Map[line.Ha_Map<noise_level] = 0.

    # Normalizar
    line.Norm_Ha = 0
    for i in range(len(line.Order_Prob_List)):
        line.Norm_Ha = line.Norm_Ha + line.Order_Prob_List[i]

    line.Order_Prob_List = line.Order_Prob_List / line.Norm_Ha
    line.Ha_Map = line.Ha_Map / line.Norm_Ha

    # Calculate P
    line.cumulative_P_values = np.cumsum(line.Order_Prob_List)

    # Calculate P map
    line.cumulative_Ha_Map = np.interp(line.Ha_Map, line.Order_Prob_List, line.cumulative_P_values)

    line.Prob_Val_Ha = 0
    line.Cumulative_Prob_Ha = 0
    square = np.arange(3)-1

    for j in square:
        for i in square:
            # The probability of Ha (p)
            line.Prob_Val_Ha = line.Prob_Val_Ha + line.Ha_Map[Pix_Y+j,Pix_X+i]
            # The cumulative probability of Ha (P)
            line.Cumulative_Prob_Ha = line.Cumulative_Prob_Ha + line.cumulative_Ha_Map[Pix_Y+j,Pix_X+i]

    line.Prob_Val_Ha = line.Prob_Val_Ha / 9
    line.Cumulative_Prob_Ha = line.Cumulative_Prob_Ha/9

    # ------------- Save Data ------------------------------------------------------------------------------------

    # SN type
    All_SN_type[Galaxy] = SN_TYPE
    # SN Ia and cumulative SN Ia Probability
    All_Prob_Ha[Galaxy] = line.Prob_Val_Ha
    All_Cumul_Ha[Galaxy] = line.Cumulative_Prob_Ha

    data_Ha = np.column_stack((All_SN_type,All_Prob_Ha,All_Cumul_Ha))
    np.savetxt('../Results_Ha.txt',data_Ha)

ASASSN13an


/home/asier/anaconda2/lib/python2.7/site-packages/astropy/convolution/convolve.py:108: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .boundary_none import (convolve1d_boundary_none,
/home/asier/anaconda2/lib/python2.7/site-packages/astropy/convolution/convolve.py:112: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .boundary_extend import (convolve1d_boundary_extend,
/home/asier/anaconda2/lib/python2.7/site-packages/astropy/convolution/convolve.py:116: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .boundary_fill import (convolve1d_boundary_fill,
/home/asier/anaconda2/lib/python2.7/site-packages/astropy/convolution/convolve.py:120: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .boundary_wrap import (convolve1d_boundary_wrap,


ASASSN13ch
ASASSN13cj
ASASSN13cp
ASASSN13dm
ASASSN14at
ASASSN14ba
ASASSN14bf
ASASSN14co
ASASSN14cu
ASASSN14db
ASASSN14dz
ASASSN14hr
ASASSN14ig
ASASSN14ko
ASASSN14lt
ASASSN14lu
ASASSN14lv
ASASSN15aj
CSP14aaq


a floating-point value was expected. [astropy.wcs.wcs]


SN1970A
SN1985G
SN1990Q
SN1997dn
SN1998bw
SN2001fv
SN2004ef
SN2004gs
SN2005ag
SN2005bg
SN2005hc
SN2005ku
SN2005M
SN2005na
SN2006br
SN2006D
SN2006ej
SN2006et
SN2006hx
SN2006lu
SN2006my
SN2006os
SN2006ot
SN2007al
SN2007ba
SN2007bc
SN2007cg
SN2007hj
SN2007ol
SN2009bb
SN2009I
SN2010F
SN2012gm
SN2012ho
SN2014cd
